# Pandas - Alteração e Transformação os Dados

### Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import pandera as pa
import datetime

### Configuração do ambiente

In [2]:
pd.options.display.float_format = '{:20,.2f}'.format
pd.option_context('display.precision', 2)

## Dataframes de Exemplo

### Criação de um Dataframe do zero

In [3]:
# Criando o numpy.array (data) bidimensional
data = np.array([['Analista de Contabilidade', 'Contabilidade', 10000.00],
                ['Analista de Contabilidade', 'Contabilidade', 10000.00],
                ['Engenheiro Civil', 'Engenharia', 15000.00],
                ['Desenvolvedor', 'TI', 11000.00],
                ['Gerente de TI', 'TI',  15000.0],
                ['Analista de BI', 'TI', 13000.0],
                ['DBA', 'TI', 10500.00],
                ['Analista Financeiro', 'Finanças', 10500.00]])

#Criando o DataFrame com o np.array (data) com as colunas ('Cargo', 'Area', 'Salario'), indexado de 100 até 108
dfc = pd.DataFrame(data, index=range(100,108),columns=['Cargo', 'Area', 'Salario'])
dfc

,Cargo,Area,Salario
100,Analista de Contabilidade,Contabilidade,10000.0
101,Analista de Contabilidade,Contabilidade,10000.0
102,Engenheiro Civil,Engenharia,15000.0
103,Desenvolvedor,TI,11000.0
104,Gerente de TI,TI,15000.0
105,Analista de BI,TI,13000.0
106,DBA,TI,10500.0
107,Analista Financeiro,Finanças,10500.0


### Outra maneira

In [4]:
data = {'Cargo':  ['Analista de Contabilidade','Analista de Contabilidade','Engenheiro Civil','Desenvolvedor','Gerente de TI', 'Analista de BI','DBA','Analista Financeiro'],
        'Area': ['Contabilidade','Contabilidade','Engenharia','TI','TI','TI','TI','Finanças'],
        'Salario': [10000.00,10000.00,15000.00,11000.00, np.nan, np.nan, 10500.00, 10500.00]
       }
dfcz = pd.DataFrame(data,index=range(100,108), columns=['Cargo','Area','Salario'])

dfcz

,Cargo,Area,Salario
100,Analista de Contabilidade,Contabilidade,"10,000.00"
101,Analista de Contabilidade,Contabilidade,"10,000.00"
102,Engenheiro Civil,Engenharia,"15,000.00"
103,Desenvolvedor,TI,"11,000.00"
104,Gerente de TI,TI,NaN
105,Analista de BI,TI,NaN
106,DBA,TI,"10,500.00"
107,Analista Financeiro,Finanças,"10,500.00"


### Carga de Arquivo CSV - Com validações
 - sep = Separador
 - parse_dates = Avalia e transforma em um campo data
 - dayfirst = Informa que os campos datas tem na primeira parte o valor do dia
 - na_values = Trasforma os valores identificados nesta lista em valores nulos
 - decimal = Caracter para reconhecer o ponto decimal 
 
 e validação do schema com a biblioteca **Pandera**

In [5]:
valores_ausentes = ['**','***','###!','####','****','*****','NULL'] 
dfo = pd.read_csv("Datasets\ocorrencia_2010_2020.csv", sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes) 
schema = pa.DataFrameSchema( 
    columns = { 
        "codigo_ocorrencia": pa.Column(pa.Int), 
        "codigo_ocorrencia2": pa.Column(pa.Int), 
        "ocorrencia_classificacao": pa.Column(pa.String), 
        "ocorrencia_cidade": pa.Column(pa.String), 
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True), 
        "ocorrencia_aerodromo": pa.Column(pa.String, nullable=True), 
        "ocorrencia_dia": pa.Column(pa.DateTime), 
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches( 
               r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True), 
        "total_recomendacoes": pa.Column(pa.Int), 
        "obs": pa.Column(pa.String, required=False) 
    }    
)

schema.validate(dfo)


,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5747,79804,79804,79804,79804,79804,INCIDENTE,-23.0069444444,-47.1344444444,CAMPINAS,SP,...,2020-12-29,19:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
5748,79757,79757,79757,79757,79757,INCIDENTE GRAVE,-10.804722,-49.756389,LAGOA DA CONFUSÃO,TO,...,2020-12-30,18:30:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
5749,79802,79802,79802,79802,79802,INCIDENTE,-22.81,-43.25055555555,RIO DE JANEIRO,RJ,...,2020-12-30,00:54:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
5750,79756,79756,79756,79756,79756,INCIDENTE GRAVE,-22.507222,-54.364167,VICENTINA,MS,...,2020-12-31,09:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO


### Mesmo Dataframe, porém com os valores inconsistentes

In [6]:
dfoz = pd.read_csv("Datasets\ocorrencia_2010_2020.csv", sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True) 
dfoz

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5747,79804,79804,79804,79804,79804,INCIDENTE,-23.0069444444,-47.1344444444,CAMPINAS,SP,...,2020-12-29,19:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
5748,79757,79757,79757,79757,79757,INCIDENTE GRAVE,-10.804722,-49.756389,LAGOA DA CONFUSÃO,TO,...,2020-12-30,18:30:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
5749,79802,79802,79802,79802,79802,INCIDENTE,-22.81,-43.25055555555,RIO DE JANEIRO,RJ,...,2020-12-30,00:54:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
5750,79756,79756,79756,79756,79756,INCIDENTE GRAVE,-22.507222,-54.364167,VICENTINA,MS,...,2020-12-31,09:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO


### Carga de Arquivo CSV

#### Com data e hora

In [7]:
dfr = pd.read_csv('Datasets/reviews.csv',parse_dates=['review_creation_date','review_answer_timestamp'])
dfr

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99995,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09,2017-12-11 20:06:42
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22,2018-03-23 09:10:43
99997,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01,2018-07-02 12:59:13
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15,2017-12-16 01:29:43


#### Com casas decimais

In [8]:
ds = pd.read_csv('Datasets\Arquivo_Salarios_Colaboradores_2020.csv',sep=';', decimal=',')
ds.head()

,Ordem,salario,idade,tempocasa,escolar,qproj_estra,proj_sustent,proj_6sigma,proj_social,notaavalia
0,1,"8,000.80",25,4,11,1,1,1,0,79.38
1,2,"8,500.17",24,5,11,0,0,1,0,84.13
2,3,"3,350.59",22,1,12,0,0,0,0,46.15
3,4,"9,500.24",28,4,14,1,0,0,1,83.85
4,5,"1,500.63",12,2,12,0,0,0,1,73.64


# Transformando os dados do Data Frame

### Alterando o tipo de dado de uma coluna

In [9]:
dfr["review_score_str"] = dfr["review_score"].astype("object")
dfr.dtypes

review_id                          object
order_id                           object
review_score                        int64
review_comment_title               object
review_comment_message             object
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
review_score_str                   object
dtype: object

### Calculando a diferença de dias

In [10]:
dfr["diferenca_dias"] = dfr["review_creation_date"] - dfr["review_creation_date"].min()
dfr.loc[:,['review_creation_date','diferenca_dias']]

,review_creation_date,diferenca_dias
0,2018-01-18,473 days
1,2018-03-10,524 days
2,2018-02-17,503 days
3,2017-04-21,201 days
4,2018-03-01,515 days
...,...,...
99995,2017-12-09,433 days
99996,2018-03-22,536 days
99997,2018-07-01,637 days
99998,2017-12-15,439 days


### Criando a coluna de trimestre

In [11]:
dfr["trimestre_review"] = dfr["review_creation_date"].dt.quarter
dfr.loc[:,['review_creation_date','diferenca_dias', 'trimestre_review']]

,review_creation_date,diferenca_dias,trimestre_review
0,2018-01-18,473 days,1
1,2018-03-10,524 days,1
2,2018-02-17,503 days,1
3,2017-04-21,201 days,2
4,2018-03-01,515 days,1
...,...,...,...
99995,2017-12-09,433 days,4
99996,2018-03-22,536 days,1
99997,2018-07-01,637 days,3
99998,2017-12-15,439 days,4


### Criando uma nova coluna com o ano

In [12]:
dfr["ano_review"] = dfr["review_creation_date"].dt.year

### Extraindo o mês e o dia

In [13]:
dfr["mes_review"], dfr["dia_review"] = (dfr["review_creation_date"].dt.month, dfr["review_creation_date"].dt.day)
dfr.loc[:,['review_creation_date','diferenca_dias', 'trimestre_review','ano_review','mes_review','dia_review']]

,review_creation_date,diferenca_dias,trimestre_review,ano_review,mes_review,dia_review
0,2018-01-18,473 days,1,2018,1,18
1,2018-03-10,524 days,1,2018,3,10
2,2018-02-17,503 days,1,2018,2,17
3,2017-04-21,201 days,2,2017,4,21
4,2018-03-01,515 days,1,2018,3,1
...,...,...,...,...,...,...
99995,2017-12-09,433 days,4,2017,12,9
99996,2018-03-22,536 days,1,2018,3,22
99997,2018-07-01,637 days,3,2018,7,1
99998,2017-12-15,439 days,4,2017,12,15


## Alteração de valores de uma coluna, com base em uma condição

Neste exemplo, alterando os dados da coluna ocorrencia_aerodromo

In [14]:
dfo.loc[dfo.ocorrencia_uf == 'SP',['ocorrencia_aerodromo']] = 'GRAVE'

## Atribuição de valores não informados a valores inválidos e substitui pelo padrão nulo do pandas

In [15]:
dfoz.loc[dfoz.divulgacao_relatorio_numero == '***',['divulgacao_relatorio_numero']] = pd.NA
dfoz

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5747,79804,79804,79804,79804,79804,INCIDENTE,-23.0069444444,-47.1344444444,CAMPINAS,SP,...,2020-12-29,19:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
5748,79757,79757,79757,79757,79757,INCIDENTE GRAVE,-10.804722,-49.756389,LAGOA DA CONFUSÃO,TO,...,2020-12-30,18:30:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
5749,79802,79802,79802,79802,79802,INCIDENTE,-22.81,-43.25055555555,RIO DE JANEIRO,RJ,...,2020-12-30,00:54:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
5750,79756,79756,79756,79756,79756,INCIDENTE GRAVE,-22.507222,-54.364167,VICENTINA,MS,...,2020-12-31,09:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO


### Substituindo valores em escala com replace, não importando o nome da coluna, ele faz de todas que encontrar os valores e substitui pelo padrão nulo do pandas


In [16]:
dfozi = dfoz.replace(['**','***','###!','####','****','*****','NULL'],pd.NA)
dfozi

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,<NA>,<NA>,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5747,79804,79804,79804,79804,79804,INCIDENTE,-23.0069444444,-47.1344444444,CAMPINAS,SP,...,2020-12-29,19:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
5748,79757,79757,79757,79757,79757,INCIDENTE GRAVE,-10.804722,-49.756389,LAGOA DA CONFUSÃO,TO,...,2020-12-30,18:30:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
5749,79802,79802,79802,79802,79802,INCIDENTE,-22.81,-43.25055555555,RIO DE JANEIRO,RJ,...,2020-12-30,00:54:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
5750,79756,79756,79756,79756,79756,INCIDENTE GRAVE,-22.507222,-54.364167,VICENTINA,MS,...,2020-12-31,09:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO


## Assim podemos utilizar o comando abaixo para verificar quais colunas tem e quantos registros estão com o valor nulo

In [17]:
dfoz.isna().sum()

codigo_ocorrencia                    0
codigo_ocorrencia1                   0
codigo_ocorrencia2                   0
codigo_ocorrencia3                   0
codigo_ocorrencia4                   0
ocorrencia_classificacao             0
ocorrencia_latitude               1565
ocorrencia_longitude              1565
ocorrencia_cidade                    0
ocorrencia_uf                        0
ocorrencia_pais                      0
ocorrencia_aerodromo                 0
ocorrencia_dia                       0
ocorrencia_hora                      1
investigacao_aeronave_liberada     341
investigacao_status                340
divulgacao_relatorio_numero       3793
divulgacao_relatorio_publicado       0
divulgacao_dia_publicacao         4258
total_recomendacoes                  0
total_aeronaves_envolvidas           0
ocorrencia_saida_pista               0
dtype: int64

### ou

In [18]:
dfoz.isnull().sum()

codigo_ocorrencia                    0
codigo_ocorrencia1                   0
codigo_ocorrencia2                   0
codigo_ocorrencia3                   0
codigo_ocorrencia4                   0
ocorrencia_classificacao             0
ocorrencia_latitude               1565
ocorrencia_longitude              1565
ocorrencia_cidade                    0
ocorrencia_uf                        0
ocorrencia_pais                      0
ocorrencia_aerodromo                 0
ocorrencia_dia                       0
ocorrencia_hora                      1
investigacao_aeronave_liberada     341
investigacao_status                340
divulgacao_relatorio_numero       3793
divulgacao_relatorio_publicado       0
divulgacao_dia_publicacao         4258
total_recomendacoes                  0
total_aeronaves_envolvidas           0
ocorrencia_saida_pista               0
dtype: int64

### ou com Lambda

In [19]:
dfoz.apply(lambda x: sum(x.isnull()),axis=0)

codigo_ocorrencia                    0
codigo_ocorrencia1                   0
codigo_ocorrencia2                   0
codigo_ocorrencia3                   0
codigo_ocorrencia4                   0
ocorrencia_classificacao             0
ocorrencia_latitude               1565
ocorrencia_longitude              1565
ocorrencia_cidade                    0
ocorrencia_uf                        0
ocorrencia_pais                      0
ocorrencia_aerodromo                 0
ocorrencia_dia                       0
ocorrencia_hora                      1
investigacao_aeronave_liberada     341
investigacao_status                340
divulgacao_relatorio_numero       3793
divulgacao_relatorio_publicado       0
divulgacao_dia_publicacao         4258
total_recomendacoes                  0
total_aeronaves_envolvidas           0
ocorrencia_saida_pista               0
dtype: int64

### Este comando serve para uma coluna específica também.

In [20]:
dfoz.ocorrencia_latitude.isna().sum()

1565

### ou

In [21]:
dfoz.ocorrencia_latitude.isnull().sum()

1565

### Depois podemos preencher todos, com o comando fillna ou replace

In [22]:
dfozteste = dfoz.fillna(10)
dfozteste.isnull().sum()

codigo_ocorrencia                 0
codigo_ocorrencia1                0
codigo_ocorrencia2                0
codigo_ocorrencia3                0
codigo_ocorrencia4                0
ocorrencia_classificacao          0
ocorrencia_latitude               0
ocorrencia_longitude              0
ocorrencia_cidade                 0
ocorrencia_uf                     0
ocorrencia_pais                   0
ocorrencia_aerodromo              0
ocorrencia_dia                    0
ocorrencia_hora                   0
investigacao_aeronave_liberada    0
investigacao_status               0
divulgacao_relatorio_numero       0
divulgacao_relatorio_publicado    0
divulgacao_dia_publicacao         0
total_recomendacoes               0
total_aeronaves_envolvidas        0
ocorrencia_saida_pista            0
dtype: int64

### ou

In [23]:
dfozteste = dfoz.replace([pd.NA], 10)
dfozteste.isnull().sum()

codigo_ocorrencia                 0
codigo_ocorrencia1                0
codigo_ocorrencia2                0
codigo_ocorrencia3                0
codigo_ocorrencia4                0
ocorrencia_classificacao          0
ocorrencia_latitude               0
ocorrencia_longitude              0
ocorrencia_cidade                 0
ocorrencia_uf                     0
ocorrencia_pais                   0
ocorrencia_aerodromo              0
ocorrencia_dia                    0
ocorrencia_hora                   0
investigacao_aeronave_liberada    0
investigacao_status               0
divulgacao_relatorio_numero       0
divulgacao_relatorio_publicado    0
divulgacao_dia_publicacao         0
total_recomendacoes               0
total_aeronaves_envolvidas        0
ocorrencia_saida_pista            0
dtype: int64

### Ou podemos preencher somente de uma coluna

In [24]:
dfoz_teste = dfoz.fillna(value={'divulgacao_dia_publicacao':10})
dfoz_teste.isnull().sum()

codigo_ocorrencia                    0
codigo_ocorrencia1                   0
codigo_ocorrencia2                   0
codigo_ocorrencia3                   0
codigo_ocorrencia4                   0
ocorrencia_classificacao             0
ocorrencia_latitude               1565
ocorrencia_longitude              1565
ocorrencia_cidade                    0
ocorrencia_uf                        0
ocorrencia_pais                      0
ocorrencia_aerodromo                 0
ocorrencia_dia                       0
ocorrencia_hora                      1
investigacao_aeronave_liberada     341
investigacao_status                340
divulgacao_relatorio_numero       3793
divulgacao_relatorio_publicado       0
divulgacao_dia_publicacao            0
total_recomendacoes                  0
total_aeronaves_envolvidas           0
ocorrencia_saida_pista               0
dtype: int64

### Ou podemos excluir a linha com null de uma determinada coluna

In [25]:
dfoz_teste = dfoz.dropna(subset=['ocorrencia_latitude'])
dfoz_teste.isnull().sum()

codigo_ocorrencia                    0
codigo_ocorrencia1                   0
codigo_ocorrencia2                   0
codigo_ocorrencia3                   0
codigo_ocorrencia4                   0
ocorrencia_classificacao             0
ocorrencia_latitude                  0
ocorrencia_longitude                 0
ocorrencia_cidade                    0
ocorrencia_uf                        0
ocorrencia_pais                      0
ocorrencia_aerodromo                 0
ocorrencia_dia                       0
ocorrencia_hora                      1
investigacao_aeronave_liberada     341
investigacao_status                313
divulgacao_relatorio_numero       2233
divulgacao_relatorio_publicado       0
divulgacao_dia_publicacao         2693
total_recomendacoes                  0
total_aeronaves_envolvidas           0
ocorrencia_saida_pista               0
dtype: int64

### Incluindo linhas no Dataframe

#### Incluindo linhas com valores nulos no inicio

In [26]:
dfc.loc[-1] = [np.nan,np.nan,np.nan]  # adionando com valores nulos
dfc.index = dfc.index + 1  # shifting index
dfc = dfc.sort_index()  # sorting by index
dfc

,Cargo,Area,Salario
0,NaN,NaN,NaN
101,Analista de Contabilidade,Contabilidade,10000.0
102,Analista de Contabilidade,Contabilidade,10000.0
103,Engenheiro Civil,Engenharia,15000.0
104,Desenvolvedor,TI,11000.0
105,Gerente de TI,TI,15000.0
106,Analista de BI,TI,13000.0
107,DBA,TI,10500.0
108,Analista Financeiro,Finanças,10500.0


#### Incluindo linhas com valores nulos no fim

In [27]:
dfc.loc[dfc.index.max()+1] = [np.nan,np.nan,np.nan]  # adionando com valores nulos
dfc

,Cargo,Area,Salario
0,NaN,NaN,NaN
101,Analista de Contabilidade,Contabilidade,10000.0
102,Analista de Contabilidade,Contabilidade,10000.0
103,Engenheiro Civil,Engenharia,15000.0
104,Desenvolvedor,TI,11000.0
105,Gerente de TI,TI,15000.0
106,Analista de BI,TI,13000.0
107,DBA,TI,10500.0
108,Analista Financeiro,Finanças,10500.0
109,NaN,NaN,NaN


### Apagando as linhas com todos os valores nulos

In [28]:
dfc.dropna(how="all", inplace=True)
dfc

,Cargo,Area,Salario
101,Analista de Contabilidade,Contabilidade,10000.0
102,Analista de Contabilidade,Contabilidade,10000.0
103,Engenheiro Civil,Engenharia,15000.0
104,Desenvolvedor,TI,11000.0
105,Gerente de TI,TI,15000.0
106,Analista de BI,TI,13000.0
107,DBA,TI,10500.0
108,Analista Financeiro,Finanças,10500.0


### Apagando as linhas com valores nulos

In [29]:
dfoz_nulos = dfoz_teste.copy()
dfoz_nulos.dropna(inplace=True)
dfoz_nulos.isnull().sum()

codigo_ocorrencia                 0
codigo_ocorrencia1                0
codigo_ocorrencia2                0
codigo_ocorrencia3                0
codigo_ocorrencia4                0
ocorrencia_classificacao          0
ocorrencia_latitude               0
ocorrencia_longitude              0
ocorrencia_cidade                 0
ocorrencia_uf                     0
ocorrencia_pais                   0
ocorrencia_aerodromo              0
ocorrencia_dia                    0
ocorrencia_hora                   0
investigacao_aeronave_liberada    0
investigacao_status               0
divulgacao_relatorio_numero       0
divulgacao_relatorio_publicado    0
divulgacao_dia_publicacao         0
total_recomendacoes               0
total_aeronaves_envolvidas        0
ocorrencia_saida_pista            0
dtype: int64

### Preencher os valores nulos com outros padrões

### Exemplo com o valor mínimo do campo

In [30]:
dfcx = dfcz
dfcx['SalarioMin'] = dfcx['Salario'].fillna(dfcx['Salario'].min())
dfcx

,Cargo,Area,Salario,SalarioMin
100,Analista de Contabilidade,Contabilidade,"10,000.00","10,000.00"
101,Analista de Contabilidade,Contabilidade,"10,000.00","10,000.00"
102,Engenheiro Civil,Engenharia,"15,000.00","15,000.00"
103,Desenvolvedor,TI,"11,000.00","11,000.00"
104,Gerente de TI,TI,NaN,"10,000.00"
105,Analista de BI,TI,NaN,"10,000.00"
106,DBA,TI,"10,500.00","10,500.00"
107,Analista Financeiro,Finanças,"10,500.00","10,500.00"


### Exemplo com o média do valor do campo

In [31]:
dfcx = dfcz
dfcx['SalarioMedia'] = dfcx['Salario'].fillna(dfcx['Salario'].mean())
dfcx

,Cargo,Area,Salario,SalarioMin,SalarioMedia
100,Analista de Contabilidade,Contabilidade,"10,000.00","10,000.00","10,000.00"
101,Analista de Contabilidade,Contabilidade,"10,000.00","10,000.00","10,000.00"
102,Engenheiro Civil,Engenharia,"15,000.00","15,000.00","15,000.00"
103,Desenvolvedor,TI,"11,000.00","11,000.00","11,000.00"
104,Gerente de TI,TI,NaN,"10,000.00","11,166.67"
105,Analista de BI,TI,NaN,"10,000.00","11,166.67"
106,DBA,TI,"10,500.00","10,500.00","10,500.00"
107,Analista Financeiro,Finanças,"10,500.00","10,500.00","10,500.00"


### Para padronizar dados categóricos com a Moda, isto é com o valor que mais se repete na base

In [32]:
dfoz['investigacao_status_moda'] = dfoz['investigacao_status'].fillna(dfoz['investigacao_status'].mode()[0])
dfoz.loc[:,['investigacao_status','investigacao_status_moda']].isnull().sum()

investigacao_status         340
investigacao_status_moda      0
dtype: int64

### Usando Lambda para preencher um valor no Dataframe

In [33]:
dfcx['Salario_Hora'] = dfcx.SalarioMedia.apply(lambda x: x/220)
dfcx

,Cargo,Area,Salario,SalarioMin,SalarioMedia,Salario_Hora
100,Analista de Contabilidade,Contabilidade,"10,000.00","10,000.00","10,000.00",45.45
101,Analista de Contabilidade,Contabilidade,"10,000.00","10,000.00","10,000.00",45.45
102,Engenheiro Civil,Engenharia,"15,000.00","15,000.00","15,000.00",68.18
103,Desenvolvedor,TI,"11,000.00","11,000.00","11,000.00",50.00
104,Gerente de TI,TI,NaN,"10,000.00","11,166.67",50.76
105,Analista de BI,TI,NaN,"10,000.00","11,166.67",50.76
106,DBA,TI,"10,500.00","10,500.00","10,500.00",47.73
107,Analista Financeiro,Finanças,"10,500.00","10,500.00","10,500.00",47.73


#### Usando o Lambda para transformar o campo data em um campo só com o dia mês e ano, fixando o dia em 01

In [34]:
dfr['review_answer_date'] = dfr['review_answer_timestamp'].apply(lambda x: datetime.datetime.strftime(x,'%Y-%m-01'))
dfr['review_answer_date']

0        2018-01-01
1        2018-03-01
2        2018-02-01
3        2017-04-01
4        2018-03-01
            ...    
99995    2017-12-01
99996    2018-03-01
99997    2018-07-01
99998    2017-12-01
99999    2017-07-01
Name: review_answer_date, Length: 100000, dtype: object

### Podemos excluir linhas que estão duplicadas também

In [35]:
dfcx.count()

Cargo           8
Area            8
Salario         6
SalarioMin      8
SalarioMedia    8
Salario_Hora    8
dtype: int64

In [36]:
dfc_teste = dfcx.drop_duplicates()
dfc_teste.count()

Cargo           7
Area            7
Salario         5
SalarioMin      7
SalarioMedia    7
Salario_Hora    7
dtype: int64

### Para transformar um campo em data e hora, neste exemplo, pegamos um campo do tipo data e um campo do tipo string que contem a hora

In [37]:
dfo['ocorrencia_dia_hora'] = pd.to_datetime(dfo.ocorrencia_dia.astype(str) + ' ' + dfo.ocorrencia_hora)
dfo['ocorrencia_dia_hora']

0      2010-01-03 12:00:00
1      2010-01-03 11:05:00
2      2010-01-03 03:00:00
3      2010-01-04 17:30:00
4      2010-01-05 19:25:00
               ...        
5747   2020-12-29 19:00:00
5748   2020-12-30 18:30:00
5749   2020-12-30 00:54:00
5750   2020-12-31 09:00:00
5751   2020-12-31 13:24:00
Name: ocorrencia_dia_hora, Length: 5752, dtype: datetime64[ns]

### Alterando o índice do dataframe em data para agrupamento

In [38]:
dfox = dfo 
dfox.index = dfox['ocorrencia_dia_hora']
dfox.groupby(by=[dfox.index.month, dfox.index.year]).ocorrencia_classificacao.agg([len])

len
ocorrencia_dia_hora ocorrencia_dia_hora     
 1.00               2,010.00              44
                    2,011.00              59
                    2,012.00              44
                    2,013.00              58
                    2,014.00              67
...                                      ...
12.00               2,016.00              38
                    2,017.00              28
                    2,018.00              32
                    2,019.00              40
                    2,020.00              51

[132 rows x 1 columns]

### Para identificar valores duplicados no índice

In [39]:
dfox[dfox.index.duplicated()]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_dia_hora
ocorrencia_dia_hora,,,,,,,,,,,,,,,,,,,,,
2010-06-27 17:45:00,42009,42009,42009,42009,42009,INCIDENTE,NaN,NaN,CARAUARI,AM,...,17:45:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2010-06-27 17:45:00
2010-11-18 18:00:00,43268,43268,43268,43268,43268,ACIDENTE,NaN,NaN,SORRISO,MT,...,18:00:00,NaN,FINALIZADA,ENCERRADA NO RAI,NÃO,NaN,0,1,NÃO,2010-11-18 18:00:00
2011-07-20 20:00:00,45008,45008,45008,45008,45008,INCIDENTE,NaN,NaN,BRASÍLIA,DF,...,20:00:00,NaN,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2011-07-20 20:00:00
2011-08-20 20:00:00,45012,45012,45012,45012,45012,INCIDENTE,NaN,NaN,GOIÂNIA,GO,...,20:00:00,NaN,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2011-08-20 20:00:00
2012-05-25 20:45:00,52240,52240,52240,52240,52240,INCIDENTE GRAVE,NaN,NaN,CASCAVEL,PR,...,20:45:00,SIM,FINALIZADA,IG-601/CENIPA/2014,SIM,2014-10-13,0,1,NÃO,2012-05-25 20:45:00
2013-02-16 09:20:00,47100,47100,47100,47100,47100,INCIDENTE,NaN,NaN,PIRASSUNUNGA,SP,...,09:20:00,NaN,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2013-02-16 09:20:00
2013-06-15 12:00:00,49113,49113,49113,49113,49113,INCIDENTE GRAVE,-20.0108333333,-43.9666666667,NOVA LIMA,MG,...,12:00:00,SIM,FINALIZADA,IG-129/CENIPA/2013,SIM,2014-06-16,4,1,NÃO,2013-06-15 12:00:00
2014-09-11 21:30:00,52592,52592,52592,52592,52592,INCIDENTE,NaN,NaN,PARAUAPEBAS,PA,...,21:30:00,NaN,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2014-09-11 21:30:00
2015-02-22 17:30:00,53115,53115,53115,53115,53115,ACIDENTE,4.7119444444,-60.5113888889,UIRAMUTÃ,RR,...,17:30:00,SIM,FINALIZADA,A-025/CENIPA/2015,SIM,2016-10-14,0,1,NÃO,2015-02-22 17:30:00


### Para excluir uma coluna

In [40]:
dfox = dfox.drop(['ocorrencia_dia_hora'],axis=1)
dfox

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
ocorrencia_dia_hora,,,,,,,,,,,,,,,,,,,,,
2010-01-03 12:00:00,40211,40211,40211,40211,40211,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2010-01-03 11:05:00,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2010-01-03 03:00:00,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2010-01-04 17:30:00,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
2010-01-05 19:25:00,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-29 19:00:00,79804,79804,79804,79804,79804,INCIDENTE,-23.0069444444,-47.1344444444,CAMPINAS,SP,...,2020-12-29,19:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2020-12-30 18:30:00,79757,79757,79757,79757,79757,INCIDENTE GRAVE,-10.804722,-49.756389,LAGOA DA CONFUSÃO,TO,...,2020-12-30,18:30:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2020-12-30 00:54:00,79802,79802,79802,79802,79802,INCIDENTE,-22.81,-43.25055555555,RIO DE JANEIRO,RJ,...,2020-12-30,00:54:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO


### Para recriar o índice

In [41]:
dfox = dfox.reset_index()
dfox

,ocorrencia_dia_hora,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,2010-01-03 12:00:00,40211,40211,40211,40211,40211,INCIDENTE,NaN,NaN,RIO DE JANEIRO,...,2010-01-03,12:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
1,2010-01-03 11:05:00,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,2010-01-03 03:00:00,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,2010-01-04 17:30:00,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,2010-01-05 19:25:00,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5747,2020-12-29 19:00:00,79804,79804,79804,79804,79804,INCIDENTE,-23.0069444444,-47.1344444444,CAMPINAS,...,2020-12-29,19:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
5748,2020-12-30 18:30:00,79757,79757,79757,79757,79757,INCIDENTE GRAVE,-10.804722,-49.756389,LAGOA DA CONFUSÃO,...,2020-12-30,18:30:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
5749,2020-12-30 00:54:00,79802,79802,79802,79802,79802,INCIDENTE,-22.81,-43.25055555555,RIO DE JANEIRO,...,2020-12-30,00:54:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
5750,2020-12-31 09:00:00,79756,79756,79756,79756,79756,INCIDENTE GRAVE,-22.507222,-54.364167,VICENTINA,...,2020-12-31,09:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO


## Salvar um Dataframe como CSV

In [42]:
dfc.to_csv("Datasets/cargos-idx.csv")

### Para salvar sem o índice

In [43]:
dfc.to_csv("Datasets/cargos.csv", index=False)